In [10]:
import pandas as pd
import json
import numpy as np
import sqlite3 as sq

In [3]:
# This creates a new database and connection if the db doesn't exist
con = sq.connect('movies.db')

In [4]:
con

In [7]:
# Pull table data from sqlite_master
con.execute('select * from sqlite_master').fetchall()
con.execute("select name from sqlite_master where type='table' order by name").fetchall()

[]

In [8]:
# Close Database
con.close()

In [14]:
data = json.load(open('some_movies.json'))
df = pd.json_normalize(data, sep='_')
df.head(2)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,belongs_to_collection
0,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,...,Part of the journey is the end.,Avengers: Endgame,False,8.3,12607,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN
1,False,/wcC7kCICL6x6zHUlUyNp9pWoqW1.jpg,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,Enter the World of Pandora.,Avatar,False,7.4,21000,87096.0,Avatar Collection,/nslJVsO58Etqkk17oXMuVK4gNOF.jpg,/8nCr9W7sKus2q9PLbYsnT7iCkuT.jpg,NaN


In [15]:
movieCols = ['id', 'title', 'revenue', 'budget', 'belongs_to_collection_name', 'release_date']
movies = df.loc[:, movieCols].copy()
movies.head(2)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24
1,19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10


In [16]:
movies['release_date'] = pd.to_datetime(movies['release_date'])
movies['revenue'] = movies['revenue'] / 1000000
movies['budget'] = movies['budget'] / 1000000

In [17]:
# Create Votes DF
votes = df.loc[:, ['id', 'vote_count', 'vote_average']].copy()
votes.head(2)

,id,vote_count,vote_average
0,299534,12607,8.3
1,19995,21000,7.4


In [19]:
# Create Genres DF
genres = pd.json_normalize(data, record_path='genres', meta='id', record_prefix='genre')
genres.head(2)

,genreid,genrename,id
0,12,Adventure,299534
1,878,Science Fiction,299534


In [21]:
# Create Production Company DF
productionCos = pd.json_normalize(data, record_path='production_companies', meta='id', record_prefix='production')
productionCos.head(2)

,productionid,productionlogo_path,productionname,productionorigin_country,id
0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,299534
1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,19995


In [22]:
# Reconnect to movies.db
con = sq.connect('movies.db')

In [23]:
# Add movies DF to movies.db in Movies table
movies.to_sql('Movies', con, index=False)

In [24]:
# Add votes DF to movies.db in Votes table
votes.to_sql('Votes', con, index=False)

In [25]:
# Add genres DF to movies.db in Genres table
genres.to_sql('Genres', con, index=False)

In [27]:
# Add Production Company DF to movies.db in ProdCo table
productionCos.to_sql('ProdCo', con, index=False)

In [28]:
con.execute("Select * from sqlite_master").fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "revenue" REAL,\n  "budget" REAL,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  3,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  4,
  'CREATE TABLE "Genres" (\n"genreid" INTEGER,\n  "genrename" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'ProdCo',
  'ProdCo',
  5,
  'CREATE TABLE "ProdCo" (\n"productionid" INTEGER,\n  "productionlogo_path" TEXT,\n  "productionname" TEXT,\n  "productionorigin_country" TEXT,\n  "id" INTEGER\n)')]

In [29]:
con.execute("Select name from sqlite_master where type='table' order by name").fetchall()

[('Genres',), ('Movies',), ('ProdCo',), ('Votes',)]

In [31]:
# Loading sqlite data into panda DF
pd.read_sql("Select * from Movies", con, index_col='id', parse_dates='release_date')

,title,revenue,budget,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24
19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15
299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25
597,Titanic,1845.034188,200.0,None,1997-11-18
135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06
420818,The Lion King,1656.943394,260.0,None,2019-07-12
24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25
168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01


In [36]:
# Pull average votes for every movie in 2018
con.execute("select avg(v.vote_average), a.title, a.release_date from Movies a join Votes v on a.id = v.id where a.release_date between '2018-01-01' and '2018-12-31' group by a.title, a.release_date order by avg(v.vote_average) desc").fetchall()

[(8.3, 'Avengers: Infinity War', '2018-04-25 00:00:00'),
 (7.5, 'Incredibles 2', '2018-06-14 00:00:00'),
 (7.4, 'Black Panther', '2018-02-13 00:00:00'),
 (6.5, 'Jurassic World: Fallen Kingdom', '2018-06-06 00:00:00')]

In [42]:
# What are the most successful production companies in terms of total rev
df = pd.read_sql("select p.id, p.productionname, m.revenue, m.title \
    from ProdCo p \
    left join Movies m \
    on p.id = m.id", con)

df.groupby('productionname')['revenue'].sum().sort_values(ascending=False)

productionname
Walt Disney Pictures                           9446.618940
Marvel Studios                                 9115.740912
20th Century Fox                               4632.999275
Lightstorm Entertainment                       4632.999275
Universal Pictures                             4490.220464
Lucasfilm                                      3400.763513
Paramount                                      3364.592098
Fuji Television Network                        3186.760879
Dentsu                                         3186.760879
Legendary Entertainment                        2975.172793
Amblin Entertainment                           2975.172793
Ingenious Media                                2787.965087
Dune Entertainment                             2787.965087
Walt Disney Animation Studios                  2604.983968
Bad Robot                                      2068.223624
Truenorth Productions                          2068.223624
The Kennedy/Marshall Company             

In [48]:
# Most common genres
df = pd.read_sql("select genrename, count(distinct id) from Genres group by genrename", con)
df

,genrename,count(distinct id)
0,Action,12
1,Adventure,15
2,Animation,3
3,Drama,1
4,Family,5
5,Fantasy,5
6,Romance,2
7,Science Fiction,10
8,Thriller,2


In [49]:
con.close()